In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib notebook
import matplotlib.pyplot as plt



In [105]:
pos_unigram_scores = pd.read_csv('./data/unigram-pmi-positive-scores.csv')
pos_bigram_scores = pd.read_csv('./data/bigram-pmi-positive-scores.csv')
pos_trigram_scores = pd.read_csv('./data/trigram-pmi-positive-scores.csv')

In [106]:
neg_unigram_scores = pd.read_csv('./data/unigram-pmi-negative-scores.csv')
neg_bigram_scores = pd.read_csv('./data/bigram-pmi-negative-scores.csv')
neg_trigram_scores = pd.read_csv('./data/trigram-pmi-negative-scores.csv')

#### The class for acquiriung top trigrams, bigrams and inigrams with specific threshold

In [170]:
class NgramManager:
    INVALID_SCORE = -100
    
    def __init__(self, unigram_scores, 
                 bigram_scores, 
                 trigram_scores, 
                 stop_words=None, 
                 threshold=None):
        
        self.trigram_scores = trigram_scores
        self.bigram_scores = bigram_scores
        self.unigram_scores = unigram_scores
        
        self.__threshold = threshold        
        self.__ignore_score = False
        self.__stop_words = stop_words
        
    
    def ignore_score(self, enabled):
        self.__ignore_score = enabled
        
        
    def find_important_unigrams(self, text: str):
        valuable_trigrams = self.find_importnat_ngrams_for_text(text, 
                                                          self.unigram_scores, 
                                                          (1,1))
        
        return valuable_trigrams


        
    def find_important_bigrams(self, text: str):
        valuable_trigrams = self.find_importnat_ngrams_for_text(text, 
                                                          self.bigram_scores, 
                                                          (2,2))
        
        return valuable_trigrams


    
    def find_important_trigrams(self, text: str):
        valuable_trigrams = self.find_importnat_ngrams_for_text(text, 
                                                          self.trigram_scores, 
                                                          (3,3))
        
        return valuable_trigrams
    
        
     
    def find_importnat_ngrams_for_text(self, text: str, ngram_scores, ngram_range):
        ngrams = ToNgramManager.get_all_ngrams_for_text(text, ngram_range, self.__stop_words)
        
        important_ngrams = list()
        
        for ngram in ngrams:
            score = self.find_ngram_score(ngram_scores, ngram)
            
            if score == self.INVALID_SCORE:
                continue
                
            if self.__ignore_score:
                important_ngrams.append((ngram, score))
                continue
            
            if self.__threshold is None:
                important_ngrams.append((ngram, score))
            else:
                if score >= self.__threshold:
                    important_ngrams.append((ngram, score))
                            
        important_ngrams.sort(key=lambda tup: tup[1], reverse=True)
                
        return important_ngrams

    @staticmethod
    def find_ngram_score(ngram_scores, ngram: str):
        found_df = ngram_scores[ngram_scores['ngram'] == ngram]
        if len(found_df) == 0:
            return ToNgramManager.INVALID_SCORE

        return found_df['score'].values[0] 
    
    @staticmethod
    def get_all_ngrams_for_text(text: str, ngram_range, stop_words):
        try:
            vectorizer = CountVectorizer(ngram_range=ngram_range, stop_words=stop_words)
            countvector = vectorizer.fit_transform([text])
            ngrams = vectorizer.get_feature_names()
            return ngrams
        except:
            return []

### Load dataset

In [12]:
df = pd.read_csv('./data/pos-neg-dataset.csv')

In [13]:
df.head()

,text,pos_neg_identifier
0,Ідеально для ділової поїздки! Господар зустрів...,1
1,"Затишний, чистий номер з усіма зручностями. Чу...",1
2,"При бронюванні вказала час прибуття о 7 ранку,...",0
3,"Чисто, тихо, комфортно. Зустрів і провів приєм...",1
4,На барі кава тільки 3 в 1. Хотілося звичайної ...,0


In [14]:
df['text'][0]

'Ідеально для ділової поїздки! Господар зустрів о шостій ранку, привітний. 22 поверх, вид з вікна на море. Все поряд, супермаркет, розваги, ресторани. Було пізнє виселення без доплати. Обов"зково зупинюся тут наступного разу.'

In [15]:
def get_pos_neg_text(df):
    pos, neg = [], []
    for i in range(0, len(df)):
        if df['pos_neg_identifier'][i] == 1:
            pos.append(df['text'][i])
        elif df['pos_neg_identifier'][i] == 0:
            neg.append(df['text'][i])
            
    return pos, neg
        

In [16]:
pos_texts, neg_texts = get_pos_neg_text(df)

In [17]:
pos_texts[:5]

['Ідеально для ділової поїздки! Господар зустрів о шостій ранку, привітний. 22 поверх, вид з вікна на море. Все поряд, супермаркет, розваги, ресторани. Було пізнє виселення без доплати. Обов"зково зупинюся тут наступного разу.',
 'Затишний, чистий номер з усіма зручностями. Чудовий краєвид з вікна. Є можливість підігріти їжу та зробити чай/каву.',
 'Чисто, тихо, комфортно. Зустрів і провів приємний власник. В номері холодильник, в холі мікрохвильовка, чайник. Все що, на мій погляд, необхідно',
 "Зручне розташування,чудовий вигляд з вікна,в номері є все що потрібно,зустрів приємний і вічливий чоловік.Однозначно рекомендую,обов'язково повернемось з дружиною ще раз.",
 'Нові апартаменти на останньому поверсі ЖК, біля центрального входу до Аркадії. Чисто, недорого (лютий місяць), ввічливий господар. Поруч - "Сільпо", новий ТЦ і безліч закладів харчування. До центру 20 хв. на маршрутці.']

In [18]:
neg_texts[:5]

['При бронюванні вказала час прибуття о 7 ранку, на що отримала відповідь "за наявності". Про те, що раніше 12 я заселитися не зможу, дізналася тільки коли туди приїхала. Виявляється, мені про це повідомили о 4.45 електронним листом, коли я ще спала у потязі. Вважаю, що слід попереджати хоча б за 12 годин, щоб була можливість скоригувати плани. Також мені не дали, хоча я і просила, підтвердження оплати для оформлення відрядження.',
 'На барі кава тільки 3 в 1. Хотілося звичайної кави, найкраще заварної.',
 'Немає терміналу для оплати кредиткою, тільки готівка.',
 'Нема зауважень',
 'Були проблеми з заселенням. Господар не дуже гостинний і приємний.']

In [19]:
def calc_num_of_coincided_ngram(texts, find_ngram_fn):
    count = 0
    for text in texts:
        coincided_ngrams = find_ngram_fn(text)
        if len(coincided_ngrams) > 0:
            count += 1
    return count

In [20]:
def show_statistics(tops, procents_unigrams, title):
    labels = [f"top - {top}" for top in tops]
    values = procents_unigrams.tolist()
    indexes = np.arange(len(labels))
    plt.bar(indexes, values, 0.8)
    plt.xticks(indexes, labels)
    plt.grid()
    plt.ylim(0, 100)
    plt.ylabel('%')
    plt.title(title)
    plt.show()

In [22]:
def read_stop_words(file):
    with open(file) as f:
        stop_words = f.read().split('\n')

    return stop_words

In [23]:
uk_stop_words = read_stop_words('./data/ukrainian-stopwords.txt')

In [173]:
def create_n_grams_dataset(texts, unigram_scores, bigram_score, trigram_scores, stop_words):
    ngram_mng = NgramManager(unigram_scores=unigram_scores, 
                             bigram_scores=bigram_score, 
                             trigram_scores=trigram_scores,
                            stop_words=stop_words)
    
    unigrams, bigrams, trigrams = [], [], []
    
    for text in texts:
        important_unigrams = ngram_mng.find_important_unigrams(text)
        
        if len(important_unigrams) > 0:
            unigrams.append(important_unigrams[0][0])
        else:
            unigrams.append(None)
            
            
        important_bigrams = ngram_mng.find_important_bigrams(text)
        
        if len(important_bigrams) > 0:
            bigrams.append(important_bigrams[0][0])
        else:
            bigrams.append(None)
            
        
        important_trigrams = ngram_mng.find_important_trigrams(text)
        
        if len(important_trigrams) > 0:
            trigrams.append(important_trigrams[0][0])
        else:
            trigrams.append(None)
            
            
    return unigrams, bigrams, trigrams 

### Positive data

In [179]:
pos_unigrams, pos_bigrams, pos_trigrams = create_n_grams_dataset(pos_texts, 
                                                                pos_unigram_scores,
                                                                pos_bigram_scores,
                                                                pos_trigram_scores,
                                                                uk_stop_words)

In [181]:
positive_data = pd.DataFrame.from_dict({"text": pos_texts, 
                                        'unigram': pos_unigrams,
                                        'bigram': pos_bigrams,
                                        'trigram': pos_trigrams})

In [195]:
print(f"There were found {100*sum(np.array(pos_unigrams) != None) / len(pos_texts)}% positive unigrams")

There were found 96.14860259032038% positive unigrams


In [196]:
print(f"There were found {100*sum(np.array(pos_bigrams) != None) / len(pos_texts)}% positive bigrams")

There were found 72.22903885480572% positive bigrams


In [197]:
print(f"There were found {100*sum(np.array(pos_trigrams) != None) / len(pos_texts)}% positive trigrams")

There were found 29.918200408997954% positive trigrams


In [182]:
positive_data

,text,unigram,bigram,trigram
0,Ідеально для ділової поїздки! Господар зустрів...,привітний,наступного разу,None
1,"Затишний, чистий номер з усіма зручностями. Чу...",затишний,чистий номер,затишний чистий номер
2,"Чисто, тихо, комфортно. Зустрів і провів приєм...",чисто,чисто тихо,None
3,"Зручне розташування,чудовий вигляд з вікна,в н...",вічливий,зручне розташування,зручне розташування чудовий
4,"Нові апартаменти на останньому поверсі ЖК, біл...",ввічливий,20 хв,None
5,Все,None,None,None
6,"Дуже привітний власник. Можна бути впевненим, ...",привітний,None,None
7,"Уважний та приємний господар.Зручна локація,по...",уважний,готель новий,None
8,"Сподобалось розміщення,чистота номеру.Все необ...",необхідне,None,None
9,"Хороший ремонт, красивий вид з вікна, чисто.",хороший,вид вікна,None


### Negative data

In [183]:
neg_unigrams, neg_bigrams, neg_trigrams = create_n_grams_dataset(neg_texts, 
                                                                neg_unigram_scores,
                                                                neg_bigram_scores,
                                                                neg_trigram_scores,
                                                                uk_stop_words)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [184]:
negative_data = pd.DataFrame.from_dict({"text": neg_texts, 
                                        'unigram': neg_unigrams,
                                        'bigram': neg_bigrams,
                                        'trigram': neg_trigrams})

In [198]:
print(f"There were found {100*sum(np.array(neg_unigrams) != None) / len(neg_texts)}% negative unigrams")

There were found 91.7567320021982% negative unigrams


In [199]:
print(f"There were found {100*sum(np.array(neg_bigrams) != None) / len(neg_texts)}% negative bigrams")

There were found 35.830738230445135% negative bigrams


In [200]:
print(f"There were found {100*sum(np.array(neg_trigrams) != None) / len(neg_texts)}% negative trigrams")

There were found 5.165781278622458% negative trigrams


In [186]:
negative_data

,text,unigram,bigram,trigram
0,"При бронюванні вказала час прибуття о 7 ранку,...",ранку,None,None
1,На барі кава тільки 3 в 1. Хотілося звичайної ...,хотілося,None,None
2,"Немає терміналу для оплати кредиткою, тільки г...",None,None,None
3,Нема зауважень,нема,None,None
4,Були проблеми з заселенням. Господар не дуже г...,проблеми,None,None
5,Важко знайти готель - він знаходиться в житлов...,важко,важко знайти,важко знайти готель
6,Єдиний мінус- це розташування до центру.,мінус,єдиний мінус,None
7,Кришка туалетна була зламана,None,None,None
8,"Можливо, розетки в ванній кімнаті розташовані ...",можливо,ванній кімнаті,None
9,"Температура води в басейні 24,5*С при заявлени...",води,None,None


## Example for several pos and neg texts using default threshold

In [143]:
def get_all_important_phrases(text: str, ngram_mng):
    unigrams = ngram_mng.find_important_unigrams(text)
    bigrams = ngram_mng.find_important_bigrams(text)
    trigrams = ngram_mng.find_important_trigrams(text)
    
    return unigrams, bigrams, trigrams

## positive text

In [153]:
ngram_mng = NgramManager(pos_unigram_scores, pos_bigram_scores, pos_trigram_scores, uk_stop_words, threshold=None)

In [145]:
pos_text = df['text'][127]
pos_text

'Все дуже сподобалось, чудове місцерозташування, привітний персонал, номер кращої категорії, як комплімент! Смачний сніданок! Кімната для паління! Надіємось, що ще повернемось!'

In [154]:
pos_unigrams, pos_bigrams, pos_trigrams = get_all_important_phrases(pos_text, ngram_mng)

In [155]:
pos_unigrams

[('чудове', 0.2372601427929805),
 ('привітний', 0.2318917030819313),
 ('смачний', 0.2216678982600548),
 ('місцерозташування', 0.21939314633935492),
 ('персонал', 0.2102618270580833),
 ('сподобалось', 0.11334971776608188),
 ('сніданок', 0.11029833161550498),
 ('номер', 0.06909785496356897),
 ('кімната', 0.06099226718469015)]

In [156]:
pos_bigrams

[('чудове місцерозташування', 0.24160622820224234),
 ('привітний персонал', 0.2361004592504881),
 ('персонал номер', 0.22970700490253454),
 ('смачний сніданок', 0.2248360179523493)]

In [157]:
pos_trigrams

[('місцерозташування привітний персонал', 0.2416062282022424),
 ('привітний персонал номер', 0.2416062282022424)]

## negative text

In [158]:
neg_ngram_mng = NgramManager(neg_unigram_scores, neg_bigram_scores, neg_trigram_scores, stop_words=uk_stop_words)

In [159]:
neg_unigrams, neg_bigrams, neg_trigrams = get_all_important_phrases(neg_text, neg_ngram_mng)

In [160]:
neg_unigrams

[('погана', 0.36397096282293495),
 ('шумоізоляція', 0.3421757391528649),
 ('сусіди', 0.29826514391840364),
 ('трішки', 0.25493873637388353)]

In [161]:
neg_bigrams

[('погана шумоізоляція', 0.3698932520755826)]

In [162]:
neg_trigrams

[]